# Bag of Words with LDA

Latent Dirichlet Allocation (LDA) is a model that is able to group similar data together. This will reduce the dimensionality of our data by grouping similar words into one category. For example, the words **movie**, **film**, and **show** might be grouped into one topic called **MOVIE_related**. Putting synonyms into one feature instead of numerous greatly reduces the dimensionality of our data. Hopefully this will improve our learner's score.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Load the data
train = pd.read_csv('../../data/train.tsv', sep='\t')
test = pd.read_csv('../../data/test.tsv', sep='\t')
train.shape

/home/isaac/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


(156060, 4)

## Count Vectorization

Now that the data is loaded, we can vectorize it using the Count vectorizer. This will give us a matrix of word frequencies.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

subset = train[:1000]

# Create the vectorizer
# We ignoring common english words and only look at a maximum of 500 unique words
def vectorize(phrases):   
    vectorizer = CountVectorizer(stop_words='english', min_df=2, max_df=0.95, max_features=1000)
    return vectorizer.fit_transform(phrases)

X = vectorize(subset.Phrase)
print X.shape

(1000, 272)


## LDA Dimension Reduction

Now we can put LDA to use and reduce dimensionality.

In [3]:
from sklearn.decomposition import LatentDirichletAllocation

def ldaify(X, y):
    lda = LatentDirichletAllocation(n_topics=50)
    return lda.fit_transform(X, y)

L = ldaify(X, subset.Sentiment)

## Learning

Now that we've transformed our data into categories we can begin to learn. We will compute cross validation accuracy using Random Forest, AdaBoost, and SVC models.

In [ ]:
def cv(X, y):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.svm import SVC

    forest = RandomForestClassifier(n_estimators=500)
    boost = AdaBoostClassifier()
    svc = SVC()

    from sklearn.cross_validation import cross_val_score
    import time

    t0 = time.time()
    print "Random Forest cross validation runnning..."
    forest_score = cross_val_score(forest, X, y).mean()
    print "Random Forest Score: %2.2f" % forest_score
    print "dt: %f" % (time.time() - t0)
    print ""

    t0 = time.time()
    print "AdaBoost cross validation runnning..."
    boost_score = cross_val_score(boost, X, y).mean()
    print "AdaBoost Score:      %2.2f" % boost_score
    print "dt: %f" % (time.time() - t0)
    print ""

    t0 = time.time()
    print "SVC cross validation runnning..."
    svc_score = cross_val_score(svc, X, y).mean()
    print "SVC Score:           %2.2f" % svc_score
    print "dt: %f" % (time.time() - t0)
    print ""
    
cv(L, subset.Sentiment)

Random Forest cross validation runnning...
Random Forest Score: 0.60
dt: 2.865526

AdaBoost cross validation runnning...
AdaBoost Score:      0.67
dt: 0.231300

SVC cross validation runnning...
SVC Score:           0.66
dt: 0.118748



In [ ]:
one_third_index = int(len(train) / 3)
cv = train[:one_third_index]
train = train[one_third_index:]

# Vectorize
print "vectorizing training data..."
vectorizer = CountVectorizer(stop_words='english', min_df=2, max_df=0.95)
X_train = vectorizer.fit_transform(train.Phrase)
y_train = train.Sentiment

# LDA
print "running lda transform..."
lda = LatentDirichletAllocation(n_topics=30, n_jobs=-1)
L_train = lda.fit_transform(X_train, y_train)

# Fit
print "training model..."
from sklearn.svm import SVC
svc = SVC()
svc.fit(L_train, y_train)

# predict on cross validation data
X_cv = vectorizer.transform(cv.Phrase)
L_cv = lda.transform(X_cv)

y_pred_cv = svc.predict(L_cv)
results_cv = pd.DataFrame({
    'PhraseId': cv.PhraseId,
    'Predicted': y_pred_cv,
    'Sentiment': cv.Sentiment
})
results_cv.to_csv('results_train.csv', index=False)

# Vectorize test data
print "vectorizing test data..."
X_test = vectorizer.transform(test.Phrase)
L_test = lda.transform(X_test)

# Predict
print "predicting..."
y_pred = svc.predict(L_test)

# Export results
print "exporting..."
results_test = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_pred
})

results_test.to_csv('results_test.csv', index=False)

print "done!"

vectorizing training data...
running lda transform...
